In [1]:
from langchain_core.documents import Document

# 1. Create a few sample documents about cats and dogs
documents = [
       Document(
        page_content="Cats are small, carnivorous mammals known for their agility, curiosity, and independence. They often live as domestic pets and communicate through meows, purring, and body language.",
        metadata={"topic": "cat"}
    ),
    Document(
        page_content="Dogs are loyal and social animals. They are often trained for various purposes such as companionship, herding, protection, and assistance for people with disabilities.",
        metadata={"topic": "dog"}
    ),
    Document(
        page_content="Rabbits are herbivorous mammals known for their long ears and strong hind legs. They are social creatures that live in burrows and communicate through thumping and body movements.",
        metadata={"topic": "rabbit"}
    ),
    Document(
        page_content="Apport, or fetch, is a behavior commonly trained in dogs. The dog retrieves an object such as a ball or stick and brings it back to its handler. This behavior utilizes instincts like chasing and retrieving.",
        metadata={"topic": "apport"}
    )
]

documents

[Document(metadata={'topic': 'cat'}, page_content='Cats are small, carnivorous mammals known for their agility, curiosity, and independence. They often live as domestic pets and communicate through meows, purring, and body language.'),
 Document(metadata={'topic': 'dog'}, page_content='Dogs are loyal and social animals. They are often trained for various purposes such as companionship, herding, protection, and assistance for people with disabilities.'),
 Document(metadata={'topic': 'rabbit'}, page_content='Rabbits are herbivorous mammals known for their long ears and strong hind legs. They are social creatures that live in burrows and communicate through thumping and body movements.'),
 Document(metadata={'topic': 'apport'}, page_content='Apport, or fetch, is a behavior commonly trained in dogs. The dog retrieves an object such as a ball or stick and brings it back to its handler. This behavior utilizes instincts like chasing and retrieving.')]

In [13]:
import os
from dotenv import load_dotenv
load_dotenv()
groq_api_key=os.getenv("GROQ_API_KEY")
from langchain_groq import ChatGroq
os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")

llm=ChatGroq(model="llama-3.3-70b-versatile",groq_api_key=groq_api_key)

llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002518FD1FAC0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000251939B2DA0>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
# Embeddings

from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [4]:
## Vector Store
from langchain_chroma import Chroma

vectorstore=Chroma.from_documents(documents,embeddings)
vectorstore


In [5]:
vectorstore.similarity_search("cat")

[Document(id='f3eb99ca-f900-4036-9374-49c948fd799a', metadata={'topic': 'cat'}, page_content='Cats are small, carnivorous mammals known for their agility, curiosity, and independence. They often live as domestic pets and communicate through meows, purring, and body language.'),
 Document(id='671a8d7c-1a4d-4902-9ab7-fb091407e039', metadata={'topic': 'rabbit'}, page_content='Rabbits are herbivorous mammals known for their long ears and strong hind legs. They are social creatures that live in burrows and communicate through thumping and body movements.'),
 Document(id='faa6529e-83c9-4ad1-a901-65dc4295f754', metadata={'topic': 'apport'}, page_content='Apport, or fetch, is a behavior commonly trained in dogs. The dog retrieves an object such as a ball or stick and brings it back to its handler. This behavior utilizes instincts like chasing and retrieving.'),
 Document(id='31bbbd6c-7276-4d9d-8d94-f02f1a3a4de4', metadata={'topic': 'dog'}, page_content='Dogs are loyal and social animals. They 

In [6]:
# Async query
await vectorstore.asimilarity_search("rabbits")

[Document(id='671a8d7c-1a4d-4902-9ab7-fb091407e039', metadata={'topic': 'rabbit'}, page_content='Rabbits are herbivorous mammals known for their long ears and strong hind legs. They are social creatures that live in burrows and communicate through thumping and body movements.'),
 Document(id='faa6529e-83c9-4ad1-a901-65dc4295f754', metadata={'topic': 'apport'}, page_content='Apport, or fetch, is a behavior commonly trained in dogs. The dog retrieves an object such as a ball or stick and brings it back to its handler. This behavior utilizes instincts like chasing and retrieving.'),
 Document(id='f3eb99ca-f900-4036-9374-49c948fd799a', metadata={'topic': 'cat'}, page_content='Cats are small, carnivorous mammals known for their agility, curiosity, and independence. They often live as domestic pets and communicate through meows, purring, and body language.'),
 Document(id='31bbbd6c-7276-4d9d-8d94-f02f1a3a4de4', metadata={'topic': 'dog'}, page_content='Dogs are loyal and social animals. They 

In [9]:
vectorstore.similarity_search_with_score("cat")

[(Document(id='f3eb99ca-f900-4036-9374-49c948fd799a', metadata={'topic': 'cat'}, page_content='Cats are small, carnivorous mammals known for their agility, curiosity, and independence. They often live as domestic pets and communicate through meows, purring, and body language.'),
  0.9175326824188232),
 (Document(id='671a8d7c-1a4d-4902-9ab7-fb091407e039', metadata={'topic': 'rabbit'}, page_content='Rabbits are herbivorous mammals known for their long ears and strong hind legs. They are social creatures that live in burrows and communicate through thumping and body movements.'),
  1.4678473472595215),
 (Document(id='faa6529e-83c9-4ad1-a901-65dc4295f754', metadata={'topic': 'apport'}, page_content='Apport, or fetch, is a behavior commonly trained in dogs. The dog retrieves an object such as a ball or stick and brings it back to its handler. This behavior utilizes instincts like chasing and retrieving.'),
  1.4845160245895386),
 (Document(id='31bbbd6c-7276-4d9d-8d94-f02f1a3a4de4', metadata

In [ ]:
# Retriever 1
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda


retriever=RunnableLambda(vectorstore.similarity_search).bind(k=1)
retriever.batch(["dog","cat"])

[[Document(id='31bbbd6c-7276-4d9d-8d94-f02f1a3a4de4', metadata={'topic': 'dog'}, page_content='Dogs are loyal and social animals. They are often trained for various purposes such as companionship, herding, protection, and assistance for people with disabilities.')],
 [Document(id='f3eb99ca-f900-4036-9374-49c948fd799a', metadata={'topic': 'cat'}, page_content='Cats are small, carnivorous mammals known for their agility, curiosity, and independence. They often live as domestic pets and communicate through meows, purring, and body language.')]]

In [11]:
# 2. Retriever 2
retriever=vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1}
)

retriever.batch(["dog","cat"])


[[Document(id='31bbbd6c-7276-4d9d-8d94-f02f1a3a4de4', metadata={'topic': 'dog'}, page_content='Dogs are loyal and social animals. They are often trained for various purposes such as companionship, herding, protection, and assistance for people with disabilities.')],
 [Document(id='f3eb99ca-f900-4036-9374-49c948fd799a', metadata={'topic': 'cat'}, page_content='Cats are small, carnivorous mammals known for their agility, curiosity, and independence. They often live as domestic pets and communicate through meows, purring, and body language.')]]

In [17]:
# RAG Chain

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
 Answer the question based on the context below.
 {question}
 
 context:
 {context}   

"""
prompt = ChatPromptTemplate.from_messages([("human", message)])


rag_chain = {"context": retriever, "question": RunnablePassthrough() } | prompt | llm

response = rag_chain.invoke("What is a rabbit?")

response.content


'According to the provided context, a rabbit is a herbivorous mammal characterized by its long ears and strong hind legs, known to be social creatures that live in burrows and communicate through thumping and body movements.'

In [18]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [20]:
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

In [33]:
!pip install -U langchain langchain-community langchain-openai


  Using cached langchain-1.0.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached langchain_openai-1.0.2-py3-none-any.whl.metadata (1.8 kB)
Using cached langchain-1.0.4-py3-none-any.whl (93 kB)
Using cached langchain_openai-1.0.2-py3-none-any.whl (81 kB)

  Attempting uninstall: langchain-openai

    Found existing installation: langchain-openai 1.0.1

    Uninstalling langchain-openai-1.0.1:

      Successfully uninstalled langchain-openai-1.0.1

   ---------------------------------------- 0/2 [langchain-openai]
  Attempting uninstall: langchain
   ---------------------------------------- 0/2 [langchain-openai]
    Found existing installation: langchain 1.0.3
   ---------------------------------------- 0/2 [langchain-openai]
    Uninstalling langchain-1.0.3:
   ---------------------------------------- 0/2 [langchain-openai]
      Successfully uninstalled langchain-1.0.3
   ---------------------------------------- 0/2 [langchain-openai]
   -------------------- ------------------- 1/2 [l

In [44]:
import langchain
print(langchain.__version__)


1.0.3


In [45]:
!pip install --upgrade langchain


In [50]:
!pip install langchain-community


In [56]:
# Install dependencies if needed
# !pip install langchain langchain-openai faiss-cpu tiktoken

from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 1️⃣ Create documents about cat, dog, rabbit, and apport
docs = [
    Document(page_content="Cats are small, carnivorous mammals known for agility, curiosity, and independence. They often live as domestic pets and communicate through meows, purring, and body language.", metadata={"topic": "cat"}),
    Document(page_content="Dogs are loyal, social animals trained for companionship, work, and protection. They have been bred for specific roles, from herding to assistance dogs.", metadata={"topic": "dog"}),
    Document(page_content="Rabbits are herbivorous mammals with long ears and strong hind legs. They are social creatures that communicate through thumping and body language, often living in burrows.", metadata={"topic": "rabbit"}),
    Document(page_content="Apport, or fetch, is a behavior in dogs where they retrieve an object, like a ball or stick, and bring it back to their handler. It relies on chasing and retrieving instincts.", metadata={"topic": "apport"})
]

# 2️⃣ Build a FAISS vector store and retriever
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = FAISS.from_documents(docs, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

# 3️⃣ Define helper to format retrieved docs into context
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# 4️⃣ Create a chat prompt
prompt_template = """
Answer the question based on the context below.
If the answer is not in the context, say "I don't know."

Question: {question}

Context:
{context}
"""
prompt = ChatPromptTemplate.from_messages([("human", prompt_template)])

# 5️⃣ Build the RAG chain (modern Runnable composition)
llm = ChatOpenAI(model="gpt-4o-mini")

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()   # ensures output is plain text
)

# 6️⃣ Example queries
questions = [
    "What is a rabbit?",
    "How does a dog perform apport?",
    "What makes cats different from dogs?",
    "Do rabbits live alone or in groups?"
]

for q in questions:
    answer = rag_chain.invoke(q)
    print(f"Q: {q}\nA: {answer}\n{'-'*60}")


Q: What is a rabbit?
A: A rabbit is a herbivorous mammal with long ears and strong hind legs, known for being social and communicating through thumping and body language, often living in burrows.
------------------------------------------------------------
Q: How does a dog perform apport?
A: A dog performs apport by retrieving an object, such as a ball or stick, and bringing it back to their handler, relying on their chasing and retrieving instincts.
------------------------------------------------------------
Q: What makes cats different from dogs?
A: Cats are known for their agility, curiosity, and independence, while dogs are recognized for their loyalty, social nature, and being trained for specific roles.
------------------------------------------------------------
Q: Do rabbits live alone or in groups?
A: Rabbits are social creatures and often live in groups.
------------------------------------------------------------
